In [1]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("FronkonGames/steam-games-dataset")

d:\Resource\anaconda3\envs\agent_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 83560/83560 [00:00<00:00, 262802.70 examples/s]


# 将 Hugging Face 数据集保存到本地（仅模板，不自动执行）
为避免每次从 Hugging Face 重新下载带来的耗时，可将 `ds` 保存到本地 `../data/raw/`，后续直接从本地加载。

推荐顺序（由快到慢/由保真到通用）：
- 方法 A：HF 原生 save_to_disk（最优）
  - 优点：加载最快（内存映射 Arrow），保留特征 schema 与数据类型，DatasetDict 直接保存/加载。
  - 缺点：只能用 Hugging Face Datasets 加载；占用目录且不可单文件传输。
- 方法 B：Parquet（兼顾通用与性能）
  - 优点：跨语言/跨工具通用；体积小，读写快；适合数据分析与下游 ML。
  - 缺点：分片/多文件管理；需要 pyarrow。
- 方法 C：JSONL（流式/调试友好）
  - 优点：人类可读、流式处理友好；便于小规模调试。
  - 缺点：体积大、加载慢、类型信息丢失。
- 方法 D：CSV（最通用）
  - 优点：几乎任意工具可读。
  - 缺点：体积大、类型信息丢失、慢。

你可以在下一个代码单元选择保存方式（不做业务转换，仅落盘）。

In [5]:
# 选择保存方式并保存到本地（请根据需要修改 method，再手动运行此单元）
from pathlib import Path
import os

RAW_DIR = Path("../data/raw").resolve()
RAW_DIR.mkdir(parents=True, exist_ok=True)

# 可选："hf" | "parquet" | "jsonl" | "csv"
method = "csv"  # 推荐：hf（Hugging Face save_to_disk）

# 输出位置（目录或文件前缀）
out_base = RAW_DIR / "steam_games_dataset"

# 工具函数：对 Dataset 或 DatasetDict 进行保存
def save_dataset_local(ds_obj, method: str, out_base: Path):
    method = method.lower()
    if method == "hf":
        # 保存为 Hugging Face 原生格式（目录）
        from datasets import Dataset, DatasetDict
        target_dir = out_base.with_suffix("")  # 目录
        if target_dir.exists() and any(target_dir.iterdir()):
            print(f"[skip] 目录已存在且非空：{target_dir}")
        else:
            ds_obj.save_to_disk(str(target_dir))
            print(f"[ok] 已保存到（HF 原生目录）：{target_dir}")
        print("加载示例：\nfrom datasets import load_from_disk\nds_local = load_from_disk(str(target_dir))")
    elif method == "parquet":
        # 为每个 split 导出一个 parquet 文件
        try:
            from datasets import Dataset, DatasetDict
        except Exception:
            pass
        if hasattr(ds_obj, "items"):
            for split, d in ds_obj.items():
                fp = out_base.with_name(f"{out_base.name}_{split}.parquet")
                d.to_parquet(str(fp))
                print(f"[ok] 写入：{fp}")
        else:
            fp = out_base.with_suffix(".parquet")
            ds_obj.to_parquet(str(fp))
            print(f"[ok] 写入：{fp}")
        print("加载示例（datasets）：\nfrom datasets import Dataset\nds_local = Dataset.from_parquet('path.parquet')")
    elif method == "jsonl":
        if hasattr(ds_obj, "items"):
            for split, d in ds_obj.items():
                fp = out_base.with_name(f"{out_base.name}_{split}.jsonl")
                d.to_json(str(fp), lines=True)
                print(f"[ok] 写入：{fp}")
        else:
            fp = out_base.with_suffix(".jsonl")
            ds_obj.to_json(str(fp), lines=True)
            print(f"[ok] 写入：{fp}")
        print("加载示例（pandas）：\nimport pandas as pd\ndf = pd.read_json('path.jsonl', lines=True)")
    elif method == "csv":
        if hasattr(ds_obj, "items"):
            for split, d in ds_obj.items():
                fp = out_base.with_name(f"{out_base.name}_{split}.csv")
                d.to_csv(str(fp))
                print(f"[ok] 写入：{fp}")
        else:
            fp = out_base.with_suffix(".csv")
            ds_obj.to_csv(str(fp))
            print(f"[ok] 写入：{fp}")
        print("加载示例（pandas）：\nimport pandas as pd\ndf = pd.read_csv('path.csv')")
    else:
        raise ValueError("method 必须为 'hf' | 'parquet' | 'jsonl' | 'csv'")

# 注意：本单元不会自动保存；请确认 method 后手动运行
save_dataset_local(ds, method, out_base)


Creating CSV from Arrow format: 100%|██████████| 84/84 [00:02<00:00, 31.04ba/s]

[ok] 写入：D:\Project\Agent\Game_Rec_Agent\game_rec_agent\data\raw\steam_games_dataset_train.csv
加载示例（pandas）：
import pandas as pd
df = pd.read_csv('path.csv')


In [1]:
import pandas as pd
df = pd.read_csv('..\\data\\raw\\steam_games_dataset_train.csv')

# 列字段含义说明（Steam 游戏数据集）

- AppID：Steam 应用唯一 ID（整数）。
- Name：游戏名称（字符串）。
- Release date：发行日期（字符串，格式不一，建议 `to_datetime` 解析；可能出现 "Coming soon"）。
- Estimated owners：估算拥有者数量区间（字符串，形如 "lo - hi"；通常来自第三方估算，非官方精确值）。可拆分为 `owners_lo`、`owners_hi`、`owners_mid`。
- Peak CCU：历史峰值同时在线玩家数（Concurrent Users，整数）。
- Required age：商店标注的最低年龄（整数；0 表示未标注；不等同于 ESRB/PEGI 正式评级）。
- Price：标价（字符串或数值；可能含货币符号/千分位；常见免费为 "Free" 或 0，建议清洗为 `price_num` 浮点数）。
- DLC count：对应 DLC 数量（整数）。
- About the game：商店简介/概述文案（字符串）。
- Supported languages：支持语言（列表或字符串形式；可能包含星号等标注界面/音频支持；可派生 `language_count`）。
- Average playtime two weeks：近两周平均游玩时长（分钟，数值）。
- Median playtime forever：历史总游玩时长的中位数（分钟，数值）。
- Median playtime two weeks：近两周游玩时长的中位数（分钟，数值）。
- Developers：开发商（多值；常以逗号分隔字符串或列表表示）。
- Publishers：发行商（多值；同上）。
- Categories：功能/模式类标签（如 Single-player、Multi-player、Steam Achievements 等，多值）。
- Genres：类型标签（如 Action、Indie、RPG 等，多值，粒度较粗）。
- Tags：社区标签（更细粒度，如 Roguelike、Souls-like 等，多值）。
- Screenshots：截图 URL 列表（可能为逗号分隔字符串或列表）。
- Movies：宣传/预告片等视频 URL 列表（同上）。

数据处理注意事项：
- 多标签字段（Genres/Categories/Tags/Supported languages）可能以字符串存储，需拆分或 `ast.literal_eval` 解析为列表，然后再统计频次或做共现分析。
- 价格与日期字段格式不统一，需正则提取价格数字并用 `pd.to_datetime(errors='coerce')` 解析日期。
- 近两周/总时长单位为分钟，常用于粘性/活跃度分析；对极值可使用对数或分位数裁剪处理。
- `Estimated owners` 为区间估计；分析时建议采用中位数 `owners_mid` 并结合对数坐标查看分布。

# 简单数据处理流程（可复用）

目标：在不改变语义的前提下，对常用字段进行标准化，得到一个干净、可分析/建模的表，并将结果保存到 `../data/processed/`。

步骤概览：
1) 选择关键字段并统一命名/类型（价格转数值、日期转时间、年龄转数值）。
2) 预计拥有者区间拆分（下限/上限/中位）。
3) 多标签字段标准化为列表（Genres/Categories/Tags/Supported languages）。
4) 派生便捷特征（是否免费、多人与合作标记、语言数量、发行年份）。
5) 去重并保存（优先 Parquet，缺失依赖时回落到 CSV）。

In [2]:
# 一键标准化并保存（从 df -> df_clean）
# 说明：
# - 本单元对常用字段做“轻量、可复用”的标准化清洗，便于后续分析/建模/检索。
# - 不改变原字段（尽量保留），在其基础上新增规范化列（如 price_num、owners_mid、release_year 等）。
# - 处理完成后导出到 ../data/processed/ 下（优先 Parquet，失败回退 CSV）。

from pathlib import Path
import pandas as pd
import numpy as np

# 处理后数据的输出目录
PROCESSED_DIR = Path("../data/processed").resolve()
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)


def preprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    对原始 Steam 游戏数据做标准化清洗，返回清洗后的 DataFrame。

    主要操作：
    1) 价格 Price -> 数值列 price_num（单位：货币最小单位转为小数，如 $19.99 -> 19.99；Free/0 记为 0.0）。
    2) 预计拥有者 Estimated owners -> owners_lo / owners_hi / owners_mid（将区间拆分并取中位）。
    3) 发行日期 Release date -> release_date_parsed（datetime）与 release_year（年份）。
    4) 所需年龄 Required age -> required_age（数值）。
    5) 多标签列（Genres/Categories/Tags） -> *_list 形式的 Python 列表，便于统计与共现分析。
    6) 语言 Supported languages -> language_count（语言数量；粗略统计）。
    7) 派生标记：是否免费 is_free、是否包含多人 has_multiplayer、是否包含合作 has_coop。
    8) 若存在 AppID，则对 AppID 去重（保留首条）。

    返回：仅保留原始关键列 + 新增规范化列，便于后续使用。
    """
    df = df.copy()

    # -------------------------
    # 辅助函数区域（纯函数，便于单元测试与复用）
    # -------------------------
    def parse_price(x):
        """将价格字段解析为浮点数。
        兼容："Free"/"$0.00"/"0" 等，统一返回 0.0；无法识别的返回 NaN。
        """
        if pd.isna(x):
            return np.nan
        s = str(x).strip()
        if s.lower() in {"free", "0", "0.00", "$0.00"}:
            return 0.0
        import re
        # 提取数字及最多两位小数（去掉千分位逗号）
        m = re.search(r"([0-9]+(?:\.[0-9]{1,2})?)", s.replace(",", ""))
        return float(m.group(1)) if m else np.nan

    def parse_owners_range(s):
        """将 "lo - hi" 形式的区间字符串解析为 (lo, hi) 数值元组。无法解析时返回 (NaN, NaN)。"""
        if pd.isna(s):
            return (np.nan, np.nan)
        parts = str(s).replace(",", "").split("-")
        if len(parts) == 2:
            lo = pd.to_numeric(parts[0].strip(), errors="coerce")
            hi = pd.to_numeric(parts[1].strip(), errors="coerce")
            return (lo, hi)
        return (np.nan, np.nan)

    def to_list(x):
        """将字符串或列表统一为列表。
        - 若为字符串：按逗号分隔并 strip。
        - 若为列表：直接返回。
        - 其他/缺失：返回空列表。
        """
        if pd.isna(x):
            return []
        if isinstance(x, list):
            return x
        return [t.strip() for t in str(x).split(",") if t.strip()]

    def lang_count(x):
        """粗略统计支持语言数量：
        - 若是列表：返回 len(list)
        - 若是类似 "['English','Chinese']" 的字符串：清理符号后按逗号分隔计数
        - 缺失返回 0
        注：不同数据源格式差异较大，这里仅做近似统计。
        """
        if pd.isna(x):
            return 0
        if isinstance(x, list):
            return len(x)
        s = str(x).strip()
        return len([t for t in s.replace("[","").replace("]","").replace("'","").split(",") if t.strip()])

    def has_any(labels, keywords):
        """判断 labels（列表）中是否包含 keywords 集合中的任意关键字（不区分大小写）。"""
        s = {t.lower() for t in labels}
        return any(k.lower() in s for k in keywords)

    # -------------------------
    # 核心清洗与特征派生
    # -------------------------
    # Price -> price_num（无该列则填 NaN，保持列兼容）
    if "Price" in df.columns:
        df["price_num"] = df["Price"].apply(parse_price)
    else:
        df["price_num"] = np.nan

    # Estimated owners -> owners_lo/hi/mid（以便排序、分布与对数变换）
    if "Estimated owners" in df.columns:
        owners = df["Estimated owners"].apply(parse_owners_range)
        df["owners_lo"] = owners.apply(lambda x: x[0])
        df["owners_hi"] = owners.apply(lambda x: x[1])
        df["owners_mid"] = (df["owners_lo"] + df["owners_hi"]) / 2.0

    # Release date -> 解析为 datetime 与年份，便于时间趋势分析
    if "Release date" in df.columns:
        df["release_date_parsed"] = pd.to_datetime(df["Release date"], errors="coerce")
        df["release_year"] = df["release_date_parsed"].dt.year

    # Required age -> 数值，便于分群（如青少年友好度）
    if "Required age" in df.columns:
        df["required_age"] = pd.to_numeric(df["Required age"], errors="coerce")

    # 多值列统一为列表，便于 explode/计数/共现
    for col, new_col in [
        ("Genres", "genres_list"),
        ("Categories", "categories_list"),
        ("Tags", "tags_list"),
    ]:
        if col in df.columns:
            df[new_col] = df[col].apply(to_list)
        else:
            # 若原列缺失，仍创建对应空列表列，保证下游代码健壮
            df[new_col] = [[] for _ in range(len(df))]

    # 支持语言 -> 统计语言数量（近似）
    if "Supported languages" in df.columns:
        df["language_count"] = df["Supported languages"].apply(lang_count)

    # 派生标记：是否免费 / 是否多人 / 是否合作
    df["is_free"] = df["price_num"].fillna(0) == 0
    df["has_multiplayer"] = df["categories_list"].apply(lambda L: has_any(L, ["Multi-player", "Multiplayer"]))
    df["has_coop"] = df["categories_list"].apply(lambda L: has_any(L, ["Co-op", "Cooperative"]))

    # 若存在 AppID，以 AppID 去重（同一游戏多条记录时保留第一条）
    if "AppID" in df.columns:
        before = len(df)
        df = df.drop_duplicates(subset=["AppID"], keep="first")
        after = len(df)
        print(f"[dedup] AppID 去重：{before} -> {after}")

    # 最终列选择：保留原始关键字段 + 新增规范化列（若存在即保留）
    keep_cols = [
        c for c in [
            # 原始关键字段
            "AppID","Name","Release date","Estimated owners","Required age","Price","DLC count",
            "Developers","Publishers","Genres","Categories","Tags","Supported languages","About the game",
            # 新增标准化/派生列
            "release_date_parsed","release_year",
            "owners_lo","owners_hi","owners_mid",
            "required_age",
            "price_num",
            "genres_list","categories_list","tags_list",
            "language_count",
            "is_free","has_multiplayer","has_coop",
        ] if c in df.columns
    ]
    return df[keep_cols]


# -------------------------
# 执行处理（要求上文已存在原始 df，例如：df = pd.read_csv(...)）
# -------------------------
try:
    df_clean = preprocess_df(df)
except NameError:
    # 若未定义 df，给出明确错误提示
    raise RuntimeError("未找到变量 df。请先加载原始数据到 df（例如从本地 CSV/Parquet 读入）。")

# 打印形状与样例，帮助快速自检
print("[shape] 原始 -> 清洗后：", df.shape, "->", df_clean.shape)
print("[preview]")
display(df_clean.head(3))

# -------------------------
# 持久化保存（优先 Parquet，失败回退 CSV）
# -------------------------
parquet_path = PROCESSED_DIR / "steam_games_dataset_clean.parquet"
csv_path = PROCESSED_DIR / "steam_games_dataset_clean.csv"

saved = None
try:
    # Parquet 读写更快且保留类型信息，适合中等规模数据
    df_clean.to_parquet(parquet_path, index=False)
    print(f"[save] 已保存 Parquet：{parquet_path}")
    saved = "parquet"
except Exception as e:
    # 若本机缺少 pyarrow/fastparquet 等依赖，回退到 CSV 保证流程不中断
    print(f"[warn] 保存 Parquet 失败：{e}\n改用 CSV...")
    df_clean.to_csv(csv_path, index=False)
    print(f"[save] 已保存 CSV：{csv_path}")
    saved = "csv"

# 使用建议打印
print("加载建议：")
if saved == "parquet":
    print("pandas：pd.read_parquet(path)  |  datasets：Dataset.from_parquet(path)")
else:
    print("pandas：pd.read_csv(path)")


[dedup] AppID 去重：83560 -> 83560
[shape] 原始 -> 清洗后： (83560, 39) -> (83560, 28)
[preview]


,AppID,Name,Release date,Estimated owners,Required age,Price,DLC count,Developers,Publishers,Genres,...,owners_mid,required_age,price_num,genres_list,categories_list,tags_list,language_count,is_free,has_multiplayer,has_coop
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,19.99,0,Perpetual FX Creative,Perpetual FX Creative,"Casual,Indie,Sports",...,10000.0,0,19.99,"[Casual, Indie, Sports]","[Single-player, Multi-player, Steam Achievemen...","[Indie, Casual, Sports, Bowling]",1,False,True,False
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0.99,0,Rusty Moyher,Wild Rooster,"Action,Indie",...,10000.0,0,0.99,"[Action, Indie]","[Single-player, Steam Achievements, Full contr...","[Indie, Action, Pixel Graphics, 2D, Retro, Arc...",10,False,False,False
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,4.99,0,Campião Games,Campião Games,"Action,Adventure,Indie,Strategy",...,10000.0,0,4.99,"[Action, Adventure, Indie, Strategy]",[Single-player],[],2,False,False,False


[save] 已保存 Parquet：D:\Project\Agent\Game_Rec_Agent\game_rec_agent\data\processed\steam_games_dataset_clean.parquet
加载建议：
pandas：pd.read_parquet(path)  |  datasets：Dataset.from_parquet(path)


In [3]:
print(df.columns)
print(df_clean.columns)

Index(['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU',
       'Required age', 'Price', 'DLC count', 'About the game',
       'Supported languages', 'Full audio languages', 'Reviews',
       'Header image', 'Website', 'Support url', 'Support email', 'Windows',
       'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score',
       'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations',
       'Notes', 'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')
Index(['AppID', 'Name', 'Release date', 'Estimated owners', 'Required age',
       'Price', 'DLC count', 'Developers', 'Publishers', 'Genres',
       'Categories', 'Tags', 'Supported languages', 'About the game',
       'release_date_parsed', 'release_year', 'owners_lo', 'owners_hi',
       'owners_mid', 'required_

In [4]:
ds1=pd.read_csv('..\\data\\raw\\weighted_score_above_08.csv')
ds1.shape

C:\Users\Admin\AppData\Local\Temp\ipykernel_24384\2565019102.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  ds1=pd.read_csv('..\\data\\raw\\weighted_score_above_08.csv')


(498094, 24)

In [5]:
ds1.head(2)

,recommendationid,appid,game,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,...,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location
0,147449116,10,Counter-Strike,76561199183984450,51,12,2548,0,2480,1696305457,...,1,99,2,0.889438,0,1,0,0,1,NaN
1,147374264,10,Counter-Strike,76561198099573060,226,13,2369,0,2361,1696096555,...,1,122,8,0.914834,0,1,0,0,1,NaN


In [6]:
ds2=pd.read_csv('..\\data\\raw\\steam_game\\games_description.csv')
ds2.shape

(290, 13)

In [7]:
ds2.head()

,name,short_description,long_description,genres,minimum_system_requirement,recommend_system_requirement,release_date,developer,publisher,overall_player_rating,number_of_reviews_from_purchased_people,number_of_english_reviews,link
0,Black Myth: Wukong,Black Myth: Wukong is an action RPG rooted in ...,About This Game\n\t\t\t\t\t\t\tBlack Myth: Wuk...,"['Mythology', 'Action RPG', 'Action', 'RPG', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"19 Aug, 2024",['Game Science'],['Game Science'],Overwhelmingly Positive,"(654,820)","51,931",https://store.steampowered.com/app/2358720/Bla...
1,Counter-Strike 2,"For over two decades, Counter-Strike has offer...",About This Game\n\t\t\t\t\t\t\tFor over two de...,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...","['OS: Windows® 10', 'Processor: 4 hardware CPU...","['OS: Windows® 10', 'Processor: 4 hardware CPU...","21 Aug, 2012",['Valve'],['Valve'],Very Positive,"(8,313,603)","2,258,990",https://store.steampowered.com/app/730/Counter...
2,"Warhammer 40,000: Space Marine 2",Embody the superhuman skill and brutality of a...,About This Game\nEmbody the superhuman skill a...,"['Warhammer 40K', 'Action', 'Third-Person Shoo...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"9 Sep, 2024",['Saber Interactive'],['Focus Entertainment'],Very Positive,"(81% of 62,791) All Time","51,920",https://store.steampowered.com/app/2183900/War...
3,Cyberpunk 2077,"Cyberpunk 2077 is an open-world, action-advent...",About This Game\nCyberpunk 2077 is an open-wor...,"['Cyberpunk', 'Open World', 'Nudity', 'RPG', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"10 Dec, 2020",['CD PROJEKT RED'],['CD PROJEKT RED'],Very Positive,"(680,264)","324,124",https://store.steampowered.com/app/1091500/Cyb...
4,ELDEN RING,THE CRITICALLY ACCLAIMED FANTASY ACTION RPG. R...,About This Game\nTHE CRITICALLY ACCLAIMED FANT...,"['Souls-like', 'Dark Fantasy', 'Open World', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"25 Feb, 2022","['FromSoftware, Inc.']","['FromSoftware, Inc.', 'Bandai Namco Entertain...",Very Positive,"(705,261)","491,741",https://store.steampowered.com/app/1245620/ELD...


In [8]:
ds2_rank = pd.read_csv('..\\data\\raw\\steam_game\\games_ranking.csv')
print(ds2_rank.shape)
ds2_rank.head()

(672, 4)


,game_name,genre,rank_type,rank
0,Counter-Strike 2,Action,Sales,1
1,"Warhammer 40,000: Space Marine 2",Action,Sales,2
2,Cyberpunk 2077,Action,Sales,3
3,Black Myth: Wukong,Action,Sales,4
4,ELDEN RING,Action,Sales,5


In [9]:
ds2_review = pd.read_csv('..\\data\\raw\\steam_game\\steam_game_reviews.csv')
print(ds2_review.shape)
ds2_review.head()

(992153, 8)


C:\Users\Admin\AppData\Local\Temp\ipykernel_24384\3357198403.py:1: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  ds2_review = pd.read_csv('..\\data\\raw\\steam_game\\steam_game_reviews.csv')


,review,hours_played,helpful,funny,recommendation,date,game_name,username
0,The game itself is also super fun. The PvP and...,39.9,"1,152",13,Recommended,14 September,"Warhammer 40,000: Space Marine 2",Sentinowl\n224 products in account
1,Never cared much about Warhammer until this ga...,91.5,712,116,Recommended,13 September,"Warhammer 40,000: Space Marine 2",userpig\n248 products in account
2,A salute to all the fallen battle brothers who...,43.3,492,33,Recommended,14 September,"Warhammer 40,000: Space Marine 2",Imparat0r\n112 products in account
3,this game feels like it was made in the mid 20...,16.8,661,15,Recommended,14 September,"Warhammer 40,000: Space Marine 2",Fattest_falcon
4,Reminds me of something I've lost. A genuine g...,24.0,557,4,Recommended,12 September,"Warhammer 40,000: Space Marine 2",Jek\n410 products in account


In [10]:
print(ds2.columns)
print(ds2_review.columns)
print(ds2_rank.columns)

Index(['name', 'short_description', 'long_description', 'genres',
       'minimum_system_requirement', 'recommend_system_requirement',
       'release_date', 'developer', 'publisher', 'overall_player_rating',
       'number_of_reviews_from_purchased_people', 'number_of_english_reviews',
       'link'],
      dtype='object')
Index(['review', 'hours_played', 'helpful', 'funny', 'recommendation', 'date',
       'game_name', 'username'],
      dtype='object')
Index(['game_name', 'genre', 'rank_type', 'rank'], dtype='object')


# 提取 Name 与 AppID 的一一对应字典

从 `df_clean` 中提取游戏名称与 AppID 的映射关系，构建一个不重复的字典。

In [11]:
# 从 df_clean 提取 Name 和 AppID 的一一对应字典
# 确保去重并处理可能的缺失值

# 方法 1：使用 set_index + to_dict (推荐)
# 先去重，确保每个 AppID 只对应一个 Name
name_to_appid = df_clean[['Name', 'AppID']].dropna().drop_duplicates(subset=['AppID']).set_index('Name')['AppID'].to_dict()

# 或者反向：AppID -> Name 的映射
appid_to_name = df_clean[['AppID', 'Name']].dropna().drop_duplicates(subset=['AppID']).set_index('AppID')['Name'].to_dict()

# 打印统计信息
print(f"[info] Name -> AppID 映射数量: {len(name_to_appid)}")
print(f"[info] AppID -> Name 映射数量: {len(appid_to_name)}")
print(f"[info] df_clean 总行数: {len(df_clean)}")

# 展示前几个示例
print("\n[示例] Name -> AppID (前 5 个):")
for i, (name, appid) in enumerate(list(name_to_appid.items())[:5]):
    print(f"  {name}: {appid}")

print("\n[示例] AppID -> Name (前 5 个):")
for i, (appid, name) in enumerate(list(appid_to_name.items())[:5]):
    print(f"  {appid}: {name}")

# 如果需要同时保留两个方向的映射，可以使用:
# game_mapping = {
#     'name_to_appid': name_to_appid,
#     'appid_to_name': appid_to_name
# }

[info] Name -> AppID 映射数量: 82842
[info] AppID -> Name 映射数量: 83554
[info] df_clean 总行数: 83560

[示例] Name -> AppID (前 5 个):
  Galactic Bowling: 20200
  Train Bandit: 655370
  Jolt Project: 1732930
  Henosis™: 1355720
  Two Weeks in Painland: 1139950

[示例] AppID -> Name (前 5 个):
  20200: Galactic Bowling
  655370: Train Bandit
  1732930: Jolt Project
  1355720: Henosis™
  1139950: Two Weeks in Painland


# 为 ds2、ds2_rank、ds2_review 添加 AppID 字段

使用上面构建的 `name_to_appid` 字典，通过模糊匹配将 AppID 添加到这些数据集中。

处理策略：
1. 首先尝试精确匹配
2. 如果失败，尝试标准化名称（去除特殊字符、统一大小写、去除空格等）
3. 使用模糊匹配算法（如字符串相似度）作为备选方案

In [12]:
# 构建名称标准化与模糊匹配工具
import re
from difflib import get_close_matches

def normalize_name(name):
    """
    标准化游戏名称，用于提高匹配成功率
    - 转小写
    - 去除特殊字符（保留字母数字和空格）
    - 去除多余空格
    - 去除常见后缀（如 ™, ®, ©）
    """
    if pd.isna(name):
        return ""
    
    # 转小写
    name = str(name).lower()
    
    # 去除商标符号等
    name = name.replace("™", "").replace("®", "").replace("©", "")
    
    # 去除特殊字符，只保留字母、数字、空格
    name = re.sub(r'[^a-z0-9\s]', ' ', name)
    
    # 去除多余空格
    name = ' '.join(name.split())
    
    return name.strip()


def find_appid(game_name, name_to_appid_dict, normalized_dict=None, threshold=0.85):
    """
    为游戏名称查找对应的 AppID
    
    参数:
        game_name: 要查找的游戏名称
        name_to_appid_dict: 原始的名称->AppID 字典
        normalized_dict: 标准化后的名称->AppID 字典（可选，用于加速）
        threshold: 模糊匹配的相似度阈值（0-1）
    
    返回:
        AppID（整数）或 NaN
    """
    if pd.isna(game_name):
        return np.nan
    
    # 步骤 1: 精确匹配（原始名称）
    if game_name in name_to_appid_dict:
        return name_to_appid_dict[game_name]
    
    # 步骤 2: 标准化后匹配
    normalized_name = normalize_name(game_name)
    
    if normalized_dict is None:
        # 如果没有预构建的标准化字典，现场构建
        normalized_dict = {normalize_name(k): v for k, v in name_to_appid_dict.items()}
    
    if normalized_name in normalized_dict:
        return normalized_dict[normalized_name]
    
    # 步骤 3: 模糊匹配（使用 difflib）
    # 在标准化的名称列表中查找最接近的匹配
    all_normalized_names = list(normalized_dict.keys())
    matches = get_close_matches(normalized_name, all_normalized_names, n=1, cutoff=threshold)
    
    if matches:
        matched_name = matches[0]
        return normalized_dict[matched_name]
    
    # 未找到匹配
    return np.nan


# 预构建标准化字典（提高效率）
print("[info] 构建标准化名称字典...")
normalized_name_to_appid = {normalize_name(k): v for k, v in name_to_appid.items()}
print(f"[info] 标准化字典大小: {len(normalized_name_to_appid)}")

# 测试几个示例
print("\n[测试] 标准化匹配示例:")
test_names = ["Counter-Strike", "counter strike", "COUNTER_STRIKE"]
for test_name in test_names:
    appid = find_appid(test_name, name_to_appid, normalized_name_to_appid)
    print(f"  '{test_name}' -> AppID: {appid}")

[info] 构建标准化名称字典...
[info] 标准化字典大小: 80959

[测试] 标准化匹配示例:
  'Counter-Strike' -> AppID: 10
  'counter strike' -> AppID: 10
  'COUNTER_STRIKE' -> AppID: 10


In [13]:
# 为 ds2 (games_description) 添加 AppID
print("=" * 60)
print("[处理] ds2 (games_description)")
print("=" * 60)

# 检查 ds2 中的名称列（可能是 'name' 或 'Name' 或其他）
name_col_ds2 = None
for col in ['name', 'Name', 'title', 'Title', 'game_name']:
    if col in ds2.columns:
        name_col_ds2 = col
        break

if name_col_ds2 is None:
    print(f"[警告] ds2 中未找到名称列，可用列: {list(ds2.columns)}")
    print("请手动指定名称列")
else:
    print(f"[info] 使用列: '{name_col_ds2}'")
    print(f"[info] ds2 原始形状: {ds2.shape}")
    
    # 添加 AppID 列
    ds2['AppID'] = ds2[name_col_ds2].apply(
        lambda x: find_appid(x, name_to_appid, normalized_name_to_appid)
    )
    
    # 统计匹配情况
    total = len(ds2)
    matched = ds2['AppID'].notna().sum()
    match_rate = (matched / total * 100) if total > 0 else 0
    
    print(f"[结果] 总记录数: {total}")
    print(f"[结果] 成功匹配: {matched} ({match_rate:.2f}%)")
    print(f"[结果] 未匹配: {total - matched} ({100 - match_rate:.2f}%)")
    
    # 显示一些匹配成功和失败的样例
    print("\n[样例] 匹配成功的前 3 条:")
    matched_samples = ds2[ds2['AppID'].notna()][[name_col_ds2, 'AppID']].head(3)
    for idx, row in matched_samples.iterrows():
        print(f"  '{row[name_col_ds2]}' -> AppID: {int(row['AppID'])}")
    
    print("\n[样例] 未匹配的前 3 条:")
    unmatched_samples = ds2[ds2['AppID'].isna()][[name_col_ds2]].head(3)
    for idx, row in unmatched_samples.iterrows():
        print(f"  '{row[name_col_ds2]}' -> 未找到")

print("\n" + "=" * 60)

[处理] ds2 (games_description)
[info] 使用列: 'name'
[info] ds2 原始形状: (290, 13)
[结果] 总记录数: 290
[结果] 成功匹配: 237 (81.72%)
[结果] 未匹配: 53 (18.28%)

[样例] 匹配成功的前 3 条:
  'Counter-Strike 2' -> AppID: 10
  'Cyberpunk 2077' -> AppID: 1091500
  'ELDEN RING' -> AppID: 1245620

[样例] 未匹配的前 3 条:
  'Black Myth: Wukong' -> 未找到
  'Warhammer 40,000: Space Marine 2' -> 未找到
  'Cyberpunk 2077: Phantom Liberty' -> 未找到



<bound method Series.info of 0            NaN
1           10.0
2            NaN
3      1091500.0
4      1245620.0
         ...    
285     798290.0
286    1018160.0
287     477770.0
288    1251460.0
289    1247360.0
Name: AppID, Length: 290, dtype: float64>

In [14]:
# 为 ds2_rank (games_ranking) 添加 AppID
print("=" * 60)
print("[处理] ds2_rank (games_ranking)")
print("=" * 60)

# 检查名称列
name_col_rank = None
for col in ['name', 'Name', 'title', 'Title', 'game_name']:
    if col in ds2_rank.columns:
        name_col_rank = col
        break

if name_col_rank is None:
    print(f"[警告] ds2_rank 中未找到名称列，可用列: {list(ds2_rank.columns)}")
    print("请手动指定名称列")
else:
    print(f"[info] 使用列: '{name_col_rank}'")
    print(f"[info] ds2_rank 原始形状: {ds2_rank.shape}")
    
    # 添加 AppID 列
    ds2_rank['AppID'] = ds2_rank[name_col_rank].apply(
        lambda x: find_appid(x, name_to_appid, normalized_name_to_appid)
    )
    
    # 统计匹配情况
    total = len(ds2_rank)
    matched = ds2_rank['AppID'].notna().sum()
    match_rate = (matched / total * 100) if total > 0 else 0
    
    print(f"[结果] 总记录数: {total}")
    print(f"[结果] 成功匹配: {matched} ({match_rate:.2f}%)")
    print(f"[结果] 未匹配: {total - matched} ({100 - match_rate:.2f}%)")
    
    # 显示样例
    print("\n[样例] 匹配成功的前 3 条:")
    matched_samples = ds2_rank[ds2_rank['AppID'].notna()][[name_col_rank, 'AppID']].head(3)
    for idx, row in matched_samples.iterrows():
        print(f"  '{row[name_col_rank]}' -> AppID: {int(row['AppID'])}")
    
    print("\n[样例] 未匹配的前 3 条:")
    unmatched_samples = ds2_rank[ds2_rank['AppID'].isna()][[name_col_rank]].head(3)
    for idx, row in unmatched_samples.iterrows():
        print(f"  '{row[name_col_rank]}' -> 未找到")

print("\n" + "=" * 60)

[处理] ds2_rank (games_ranking)
[info] 使用列: 'game_name'
[info] ds2_rank 原始形状: (672, 4)
[结果] 总记录数: 672
[结果] 成功匹配: 516 (76.79%)
[结果] 未匹配: 156 (23.21%)

[样例] 匹配成功的前 3 条:
  'Counter-Strike 2' -> AppID: 10
  'Cyberpunk 2077' -> AppID: 1091500
  'ELDEN RING' -> AppID: 1245620

[样例] 未匹配的前 3 条:
  'Warhammer 40,000: Space Marine 2' -> 未找到
  'Black Myth: Wukong' -> 未找到
  'DRAGON BALL: Sparking! ZERO' -> 未找到



In [15]:
# 为 ds2_review (steam_game_reviews) 添加 AppID
print("=" * 60)
print("[处理] ds2_review (steam_game_reviews)")
print("=" * 60)

# 检查名称列
name_col_review = None
for col in ['name', 'Name', 'title', 'Title', 'game_name']:
    if col in ds2_review.columns:
        name_col_review = col
        break

if name_col_review is None:
    print(f"[警告] ds2_review 中未找到名称列，可用列: {list(ds2_review.columns)}")
    print("请手动指定名称列")
else:
    print(f"[info] 使用列: '{name_col_review}'")
    print(f"[info] ds2_review 原始形状: {ds2_review.shape}")
    
    # 添加 AppID 列
    ds2_review['AppID'] = ds2_review[name_col_review].apply(
        lambda x: find_appid(x, name_to_appid, normalized_name_to_appid)
    )
    
    # 统计匹配情况
    total = len(ds2_review)
    matched = ds2_review['AppID'].notna().sum()
    match_rate = (matched / total * 100) if total > 0 else 0
    
    print(f"[结果] 总记录数: {total}")
    print(f"[结果] 成功匹配: {matched} ({match_rate:.2f}%)")
    print(f"[结果] 未匹配: {total - matched} ({100 - match_rate:.2f}%)")
    
    # 显示样例
    print("\n[样例] 匹配成功的前 3 条:")
    matched_samples = ds2_review[ds2_review['AppID'].notna()][[name_col_review, 'AppID']].head(3)
    for idx, row in matched_samples.iterrows():
        print(f"  '{row[name_col_review]}' -> AppID: {int(row['AppID'])}")
    
    print("\n[样例] 未匹配的前 3 条:")
    unmatched_samples = ds2_review[ds2_review['AppID'].isna()][[name_col_review]].head(3)
    for idx, row in unmatched_samples.iterrows():
        print(f"  '{row[name_col_review]}' -> 未找到")

print("\n" + "=" * 60)

[处理] ds2_review (steam_game_reviews)
[info] 使用列: 'game_name'
[info] ds2_review 原始形状: (992153, 8)


KeyboardInterrupt: 

# 汇总匹配结果与保存

查看所有数据集的匹配情况汇总，并可选择保存带有 AppID 的新数据集。

In [ ]:
# 汇总所有数据集的匹配情况
print("=" * 60)
print("[汇总] 所有数据集的 AppID 匹配情况")
print("=" * 60)

summary_data = []

# ds2
if 'AppID' in ds2.columns:
    total = len(ds2)
    matched = ds2['AppID'].notna().sum()
    summary_data.append({
        '数据集': 'ds2 (games_description)',
        '总记录数': total,
        '匹配成功': matched,
        '匹配率': f"{(matched/total*100):.2f}%" if total > 0 else "0%"
    })

# ds2_rank
if 'AppID' in ds2_rank.columns:
    total = len(ds2_rank)
    matched = ds2_rank['AppID'].notna().sum()
    summary_data.append({
        '数据集': 'ds2_rank (games_ranking)',
        '总记录数': total,
        '匹配成功': matched,
        '匹配率': f"{(matched/total*100):.2f}%" if total > 0 else "0%"
    })

# ds2_review
if 'AppID' in ds2_review.columns:
    total = len(ds2_review)
    matched = ds2_review['AppID'].notna().sum()
    summary_data.append({
        '数据集': 'ds2_review (steam_game_reviews)',
        '总记录数': total,
        '匹配成功': matched,
        '匹配率': f"{(matched/total*100):.2f}%" if total > 0 else "0%"
    })

summary_df = pd.DataFrame(summary_data)
print("\n")
display(summary_df)

print("\n" + "=" * 60)
print("[提示] 如需保存带有 AppID 的数据集，请运行下一个单元")
print("=" * 60)

In [ ]:
# 可选：保存带有 AppID 的数据集到本地
# 取消注释下面的代码以保存数据

from pathlib import Path

# 输出目录
OUTPUT_DIR = Path("../data/processed").resolve()
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 保存 ds2 (带 AppID)
if 'AppID' in ds2.columns:
    output_path = OUTPUT_DIR / "games_description_with_appid.csv"
    ds2.to_csv(output_path, index=False)
    print(f"[保存] ds2 已保存到: {output_path}")

# 保存 ds2_rank (带 AppID)
if 'AppID' in ds2_rank.columns:
    output_path = OUTPUT_DIR / "games_ranking_with_appid.csv"
    ds2_rank.to_csv(output_path, index=False)
    print(f"[保存] ds2_rank 已保存到: {output_path}")

# 保存 ds2_review (带 AppID)
if 'AppID' in ds2_review.columns:
    output_path = OUTPUT_DIR / "steam_game_reviews_with_appid.csv"
    ds2_review.to_csv(output_path, index=False)
    print(f"[保存] ds2_review 已保存到: {output_path}")

print("\n[完成] 所有数据集已保存到 ../data/processed/ 目录")